In [1]:
import numpy as np
import pandas as pd
import sys, os, glob

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua

In [2]:
request={
    'variable': ['air_temperature', 'relative_humidity'],
    'optional':['RASE_bias_estimate', 'humidity_bias_estimate'],
    'statid': '11035',
    'pressure_level':[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,35000,40000,45000,50000,55000,60000,65000,70000,75000,80000,85000,90000, 92500, 95000, 100000],
    'date': ['20000101', '20000101'],
#     'cdm': ['advanced_homogenisation/latitude_displacement', 'advanced_homogenisation/longitude_displacement', 'advanced_homogenisation/time_since_launch'],
    'format':'csv',
} 
df = eua.vm_request_wrapper(request, overwrite=True, vm_url='http://127.0.0.1:8007')

df['z_coordinate'] = np.log(df['z_coordinate'])
df.date_time = pd.to_datetime(df.date_time)

display(df) #= df.to_dataframe()

,RASE_bias_estimate,date_time,latitude,longitude,primary_id,report_id,z_coordinate,variable,observation_value,humidity_bias_estimate
0,NaN,2000-01-01 00:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030635,6.907755,air_temperature,205.899994,NaN
1,0.131690,2000-01-01 00:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030635,7.600902,air_temperature,205.899994,NaN
2,0.317648,2000-01-01 00:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030635,8.006368,air_temperature,206.300003,NaN
3,0.350046,2000-01-01 00:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030635,8.517193,air_temperature,210.100006,NaN
4,0.316496,2000-01-01 00:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030635,8.853665,air_temperature,213.500000,NaN
...,...,...,...,...,...,...,...,...,...,...
61,NaN,2000-01-01 12:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030637,10.819778,relative_humidity,0.828453,-0.046382
62,NaN,2000-01-01 12:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030637,11.156251,relative_humidity,0.047564,-0.036254
63,NaN,2000-01-01 12:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030637,11.350407,relative_humidity,0.760778,-0.039154
64,NaN,2000-01-01 12:00:00+00:00,48.23,16.370001,0-20001-0-11035,10000030637,11.434964,relative_humidity,0.900064,-0.025628


In [3]:
target_pressure = np.log([1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,35000,40000,45000,50000,55000,60000,65000,70000,75000,80000,85000,90000, 92500, 95000, 100000])
target_pressure

array([ 6.90775528,  7.60090246,  8.00636757,  8.51719319,  8.85366543,
        9.21034037,  9.61580548,  9.90348755, 10.1266311 , 10.30895266,
       10.46310334, 10.59663473, 10.71441777, 10.81977828, 10.91508846,
       11.00209984, 11.08214255, 11.15625052, 11.22524339, 11.28978191,
       11.35040654, 11.40756495, 11.43496392, 11.46163217, 11.51292546])

In [4]:
out = {}
for i in ['humidity_bias_estimate','relative_humidity', 'RASE_bias_estimate', 'air_temperature', 'date_time', 'latitude', 'longitude', 
          'report_id', 'primary_id', 'z_coordinate']: #, 'latitude_displacement', 'longitude_displacement', 'seconds_since_release']:
    out[i] = []
for i in df.report_id.drop_duplicates():
    i_df = df[df.report_id == i]
    hum_df = i_df[i_df.variable == 'relative_humidity']
    temp_df = i_df[i_df.variable == 'air_temperature']
    
#     out['latitude_displacement'].extend(list(np.interp(target_pressure, temp_df['z_coordinate'], temp_df['latitude_displacement'])))
#     out['longitude_displacement'].extend(list(np.interp(target_pressure, temp_df['z_coordinate'], temp_df['longitude_displacement'])))
#     out['seconds_since_release'].extend(list(np.interp(target_pressure, temp_df['z_coordinate'], temp_df['time_since_launch'])))
    
    out['humidity_bias_estimate'].extend(list(np.interp(target_pressure, hum_df['z_coordinate'], hum_df['humidity_bias_estimate'])))
    out['relative_humidity'].extend(list(np.interp(target_pressure, hum_df['z_coordinate'], hum_df['observation_value'])))
    out['RASE_bias_estimate'].extend(list(np.interp(target_pressure, temp_df['z_coordinate'], temp_df['RASE_bias_estimate'])))
    out['air_temperature'].extend(list(np.interp(target_pressure, temp_df['z_coordinate'], temp_df['observation_value'])))
    out['date_time'].extend(list([i_df['date_time'].iloc[0]]*len(target_pressure)))
    out['latitude'].extend(list([i_df['latitude'].iloc[0]]*len(target_pressure)))
    out['longitude'].extend(list([i_df['longitude'].iloc[0]]*len(target_pressure)))
    out['report_id'].extend(list([i_df['report_id'].iloc[0]]*len(target_pressure)))
    out['primary_id'].extend(list([i_df['primary_id'].iloc[0]]*len(target_pressure)))
    out['z_coordinate'].extend([1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,35000,40000,45000,50000,55000,60000,65000,70000,75000,80000,85000,90000, 92500, 95000, 100000])
out_df = pd.DataFrame.from_dict(out)
out_df

,humidity_bias_estimate,relative_humidity,RASE_bias_estimate,air_temperature,date_time,latitude,longitude,report_id,primary_id,z_coordinate
0,-0.000000,0.020402,NaN,205.899994,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,1000
1,0.013616,0.029577,0.131690,205.899994,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,2000
2,0.010815,0.030094,0.317648,206.300003,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,3000
3,0.010331,0.020739,0.350046,210.100006,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,5000
4,0.010655,0.020362,0.316496,213.500000,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,7000
5,0.010018,0.019152,0.194221,215.899994,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,10000
6,0.012309,0.028040,0.035679,216.899994,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,15000
7,0.019665,0.030437,-0.046281,218.899994,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,20000
8,0.007371,0.037673,0.061712,220.300003,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,25000
9,-0.004458,0.036647,0.067414,223.699997,2000-01-01 00:00:00+00:00,48.23,16.370001,10000030635,0-20001-0-11035,30000


In [5]:
ds = out_df.set_index('date_time').to_xarray()
display(ds)
ds.to_netcdf('test_nc_out.nc')

<xarray.Dataset>
Dimensions:                 (date_time: 50)
Coordinates:
  * date_time               (date_time) object 946684800000000000 ... 9467280...
Data variables:
    humidity_bias_estimate  (date_time) float64 -0.0 0.01362 ... 0.01424
    relative_humidity       (date_time) float64 0.0204 0.02958 ... 0.8727 0.8202
    RASE_bias_estimate      (date_time) float64 nan 0.1317 0.3176 ... 0.0 -0.0
    air_temperature         (date_time) float64 205.9 205.9 ... 270.9 273.2
    latitude                (date_time) float64 48.23 48.23 ... 48.23 48.23
    longitude               (date_time) float64 16.37 16.37 ... 16.37 16.37
    report_id               (date_time) int64 10000030635 ... 10000030637
    primary_id              (date_time) object '0-20001-0-11035' ... '0-20001...
    z_coordinate            (date_time) int64 1000 2000 3000 ... 95000 100000